In [1]:
# An implementaion of how to display all of the exchanges and market data sources that 
# are available on the CCXT trading library.

import ccxt
print(ccxt.exchanges)

['_1broker', '_1btcxe', 'acx', 'allcoin', 'anxpro', 'anybits', 'bibox', 'binance', 'bit2c', 'bitbank', 'bitbay', 'bitfinex', 'bitfinex2', 'bitflyer', 'bithumb', 'bitkk', 'bitlish', 'bitmarket', 'bitmex', 'bitsane', 'bitso', 'bitstamp', 'bitstamp1', 'bittrex', 'bitz', 'bl3p', 'bleutrade', 'braziliex', 'btcbox', 'btcchina', 'btcexchange', 'btcmarkets', 'btctradeim', 'btctradeua', 'btcturk', 'btcx', 'bxinth', 'ccex', 'cex', 'chbtc', 'chilebit', 'cobinhood', 'coinbase', 'coinbasepro', 'coincheck', 'coinegg', 'coinex', 'coinexchange', 'coinfalcon', 'coinfloor', 'coingi', 'coinmarketcap', 'coinmate', 'coinnest', 'coinone', 'coinsecure', 'coinspot', 'cointiger', 'coolcoin', 'crypton', 'cryptopia', 'deribit', 'dsx', 'ethfinex', 'exmo', 'exx', 'fcoin', 'flowbtc', 'foxbit', 'fybse', 'fybsg', 'gatecoin', 'gateio', 'gdax', 'gemini', 'getbtc', 'hadax', 'hitbtc', 'hitbtc2', 'huobi', 'huobicny', 'huobipro', 'ice3x', 'independentreserve', 'indodax', 'itbit', 'jubi', 'kraken', 'kucoin', 'kuna', 'lakebt

In [40]:
# An example of how to instantiate and load an exchange to see all of the tickers available for 
# trade on the exchange. 

import ccxt

exchange = ccxt.poloniex()
markets = exchange.load_markets() 
symbols = exchange.symbols


print(exchange.id, symbols)

poloniex ['AMP/BTC', 'ARDR/BTC', 'BCH/BTC', 'BCH/ETH', 'BCH/USDT', 'BCN/BTC', 'BCN/XMR', 'BCY/BTC', 'BLK/BTC', 'BLK/XMR', 'BTC/USDT', 'BTCD/BTC', 'BTCD/XMR', 'BTS/BTC', 'BURST/BTC', 'Bitmark/BTC', 'CLAM/BTC', 'CVC/BTC', 'CVC/ETH', 'DASH/BTC', 'DASH/USDT', 'DASH/XMR', 'DCR/BTC', 'DGB/BTC', 'DOGE/BTC', 'EMC2/BTC', 'ETC/BTC', 'ETC/ETH', 'ETC/USDT', 'ETH/BTC', 'ETH/USDT', 'EXP/BTC', 'FCT/BTC', 'FLDC/BTC', 'FLO/BTC', 'GAME/BTC', 'GAS/BTC', 'GAS/ETH', 'GNO/BTC', 'GNO/ETH', 'GNT/BTC', 'GNT/ETH', 'GRC/BTC', 'HUC/BTC', 'LBC/BTC', 'LSK/BTC', 'LSK/ETH', 'LTC/BTC', 'LTC/USDT', 'LTC/XMR', 'MAID/BTC', 'MAID/XMR', 'NAV/BTC', 'NEOS/BTC', 'NMC/BTC', 'NXC/BTC', 'NXT/BTC', 'NXT/USDT', 'NXT/XMR', 'OMG/BTC', 'OMG/ETH', 'OMNI/BTC', 'PASC/BTC', 'PINK/BTC', 'POT/BTC', 'PPC/BTC', 'RADS/BTC', 'REP/BTC', 'REP/ETH', 'REP/USDT', 'RIC/BTC', 'SBD/BTC', 'SC/BTC', 'STEEM/BTC', 'STEEM/ETH', 'STORJ/BTC', 'STRAT/BTC', 'SYS/BTC', 'VIA/BTC', 'VRC/BTC', 'VTC/BTC', 'XBC/BTC', 'XCP/BTC', 'XEM/BTC', 'XLM/BTC', 'XLM/USDT', 'XMR

In [35]:
# implementation of bid, ask and spread data based on exchanges generated from 
# the ccxt trading library system. 

import ccxt
import time

exchange = ccxt.gemini()
exchange.load_markets()
delay_seconds = exchange.rateLimit / 1000

symbols = dict()

iteration = 0 
while True: 
    
    for symbol in exchange.markets: 
        
        start_time = time.clock() 
        orderbook = exchange.fetch_order_book(symbol)
        bid = orderbook['bids'][0][0] if len(orderbook['bids']) > 0 else None
        ask = orderbook['asks'][0][0] if len(orderbook['asks']) > 0 else None
        spread = (ask - bid) if (bid and ask) else None
        market_price = {
            'bid' : bid, 
            'ask' : ask, 
            'spread' : spread
        }
        
        print(exchange.id, symbol, 'market price:\n', market_price)
        
        if iteration > 0: 
            difference = {
                'bid' : bid - symbols[symbol]['bid'], 
                'ask' : ask - symbols[symbol]['ask'],
                'spread' : spread - symbols[symbol]['spread']
            }
            
            print(difference)
            
        symbols[symbol] = market_price
        
        time.sleep(delay_seconds - (time.clock() - start_time))
        
    iteration +=1 

gemini BTC/USD market price:
 {'bid': 6601.42, 'ask': 6601.43, 'spread': 0.010000000000218279}
gemini ETH/BTC market price:
 {'bid': 0.07147, 'ask': 0.07173, 'spread': 0.00025999999999999635}
gemini ETH/USD market price:
 {'bid': 472.47, 'ask': 473.05, 'spread': 0.5799999999999841}
gemini ZEC/USD market price:
 {'bid': 170.97, 'ask': 171.77, 'spread': 0.8000000000000114}
gemini ZEC/BTC market price:
 {'bid': 0.02587, 'ask': 0.026, 'spread': 0.00012999999999999817}
gemini ZEC/ETH market price:
 {'bid': 0.3615, 'ask': 0.3676, 'spread': 0.006099999999999994}
gemini BTC/USD market price:
 {'bid': 6599.94, 'ask': 6600.43, 'spread': 0.4900000000006912}
{'bid': -1.480000000000473, 'ask': -1.0, 'spread': 0.48000000000047294}
gemini ETH/BTC market price:
 {'bid': 0.07152, 'ask': 0.07172, 'spread': 0.00020000000000000573}
{'bid': 4.999999999999449e-05, 'ask': -9.999999999996123e-06, 'spread': -5.9999999999990616e-05}
gemini ETH/USD market price:
 {'bid': 472.47, 'ask': 472.9, 'spread': 0.4299999

KeyboardInterrupt: 